In [2]:
def create_dataframe(lookback_days, total_rows, input_date,  dyn_df,  reg):
    final_df = pd.DataFrame()
    time = lookback_days
    output_date = input_date
    rows = total_rows
    fields_npi = []
    filenames = ["c1_school_closing.csv", "c2_workplace_closing.csv", "c3_cancel_public_events.csv",
            "c4_restrictions_on_gatherings.csv", "c5_close_public_transport.csv", "c6_stay_at_home_requirements.csv", "c7_movementrestrictions.csv",
            "c8_internationaltravel.csv", "h1_public_information_campaigns.csv","h2_testing_policy.csv","h3_contact_tracing.csv", "h6_facial_coverings.csv"]

    for i in filenames:
        fields_npi.append(i[:-4])
        
    count_npis = 0
    for npi_file in filenames:
        India = 0
        count_npis += 1 
        npi_df = pd.read_csv(os.path.join('timeseries', npi_file))
        # first we need to convert the header date formats to more y -m - d format
        headers = []
        for header in npi_df:
            headers.append(header)
        for i in range(3,len(headers)):
            dt = parse(headers[i])
            headers[i] = dt.strftime('%Y-%m-%d')
        npi_df.columns = headers
        for region in npi_df['country_code']:
            if region == reg:
                break           
            India += 1 

        for column in range(1, 1+time):
            new  = pd.Series([], dtype = 'float64')
            current_date = output_date - datetime.timedelta(column)
            for row in range(rows):
                new_date = current_date - datetime.timedelta(row)
                new[row] = npi_df.loc[India, str(new_date)]          

            final_df.insert(time*(count_npis-1)+column-1, fields_npi[count_npis-1][:-4] + str(column), new)

    index = pd.Series([], dtype = 'float64')
    
    current_dates = []
    for row in range(rows):
        new_date = output_date - datetime.timedelta(row)
        index[row] = new_date
        current_dates.append(new_date)
    final_df.insert(0, 'current_date', index)
    
    dyn_df1 = dyn_df[dyn_df['Code']==reg]
    dyn_df1 = dyn_df.set_index('Date', drop = True)
    #print(dyn_df1)
    for header in dyn_df1:
        if header == 'Date' or header == "Unnamed: 0" or header == 'Code':
            continue 
        new_1  = pd.Series([], dtype = 'float64')
        for row in range(rows):
                new_date = current_date - datetime.timedelta(row)
                new_1[row] = dyn_df1.loc[str(new_date), header][1] 
        

                
        final_df.insert(final_df.shape[1], header , new_1) 
        
    final_df.to_csv(os.path.join('data', 'final.csv'))      
    return final_df

In [23]:
import pandas as pd
import csv
import datetime
from dateutil.parser import parse
import os
import numpy as np
!pwd

total_rows=180
input_date=datetime.date(2020, 12, 1)
dyn_df = pd.read_csv('data/dynamic.csv')
reg="AFG"
lookback_days=60
df=create_dataframe(lookback_days, total_rows, input_date,  dyn_df,  reg)
print(df)

/home/meenakshi/pandemic/covid-xprize-master/predictors
    current_date  c1_school_clo1  c1_school_clo2  c1_school_clo3  \
0     2020-12-01             0.0             0.0             0.0   
1     2020-11-30             0.0             0.0             0.0   
2     2020-11-29             0.0             0.0             0.0   
3     2020-11-28             0.0             0.0             0.0   
4     2020-11-27             0.0             0.0             0.0   
..           ...             ...             ...             ...   
175   2020-06-09             3.0             3.0             3.0   
176   2020-06-08             3.0             3.0             3.0   
177   2020-06-07             3.0             3.0             3.0   
178   2020-06-06             3.0             3.0             3.0   
179   2020-06-05             3.0             3.0             3.0   

     c1_school_clo4  c1_school_clo5  c1_school_clo6  c1_school_clo7  \
0               0.0             0.0             0.0     

In [24]:
# countries = ['AFG', 'ARG', 'AUS', 'BGD', 'BEL', 'BRA', 'CAN', 'DNK', 'EGY', 'FRA','GHA','HKG','IND', 'IRL', 'ISR', 'ITA', 'JPN', 'KGZ', 'LTU','MEX', 'NAM','NZL', 'NGA', 'OMN', 'PER', 
#              'PRI', 'QAT', 'SAU', 'ZAF', 'KOR', 'THA', 'GBR', 'USA','ZMB']
# # countries = ['AFG','ARG','AUS','BGD','CAN']
countries = ['AUS','CAN', 'DNK','BRA', 'FRA','IND','ITA','QAT','SAU','EGY','MEX',
             'KOR','USA','ZMB']

final_df = pd.DataFrame()
for reg in countries:
    df = create_dataframe(lookback_days, total_rows, input_date,  dyn_df,  reg)
    final_df = final_df.append(df, ignore_index=True)

In [25]:
print(final_df.shape)

final_df.to_csv(os.path.join('data', 'train_final_npi.csv'))
final_df_np=final_df.to_numpy()
print(final_df_np[0,1:3])

(2520, 726)
[0.0 0.0]
